In [61]:
import os
import subprocess
import sys
import time
import pandas as pd

In [62]:
REGION = 'us-central1'
#ENDPOINT = '<YOUR_ENDPOINT>' e.g. '337dd39580cbcbd2-dot-us-central2.pipelines.googleusercontent.com'
INPUT_FILE = 'gs://input_data_amy_bkt1/input_data/Anonymized_Fermentation_Data_final.xlsx'
ARTIFACT_STORE_URI = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [63]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://workshop_trial_artifact_store_pp'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'X_train.xlsx')
TRAINING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'training')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'X_validate.xlsx')
VALIDATION_FILE_DIR='{}/{}'.format(DATA_ROOT, 'validation')
TESTING_FILE_DIR='{}/{}'.format(DATA_ROOT, 'testing')

In [9]:
max_iter=2

In [64]:
training_dataset_path=TRAINING_FILE_PATH 
validation_dataset_path=VALIDATION_FILE_PATH 
training_dataset=TRAINING_FILE_DIR 
validation_dataset=VALIDATION_FILE_DIR 
testing_dataset=TESTING_FILE_DIR 
input_file=INPUT_FILE 

import subprocess
import sys

#import fire
import pickle
import numpy as np
import pandas as pd

import json

import pickle
import time


#import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split  ## from analysis

#simple sklearn impute and scale numeric pipeline
from sklearn.pipeline import Pipeline ## from analysis
from sklearn.impute import SimpleImputer ## from analysis
from sklearn.preprocessing import StandardScaler ## from analysis
import numpy as np ## from analysis

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import RandomizedSearchCV


import functools


from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from googleapiclient import discovery
from googleapiclient import errors

In [65]:
obj = input_file
print("obj", obj)
data = pd.read_excel('gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx',sheet_name='data') 
print("Opened excel file and assigned to data")
meta_data = pd.read_excel('gs://input_data_amy_bkt1/Anonymized_Fermentation_Data_final.xlsx',sheet_name='meta data') 
print("Just after file opening")


obj gs://input_data_amy_bkt1/input_data/Anonymized_Fermentation_Data_final.xlsx
Opened excel file and assigned to data
Just after file opening


In [66]:
from sklearn.model_selection import train_test_split
#prepare data for analysis
#split out numeric from categorical varibles
var_type_filter = [x in ['independent'] for x in meta_data['variable type']]
var_dtype_filter = (data.dtypes == 'float64') | (data.dtypes == 'int64')

numeric_vars = (var_type_filter & var_dtype_filter).values
numeric_x_data = data[data.columns[numeric_vars]]



In [67]:
#things to try to predict
y_data = data[data.columns[(meta_data['target'] == 1).values]]

#meta data about variables
meta_data = meta_data.query('name in {}'.format(list(data.columns[numeric_vars].values))).set_index('name')

#Variables which will be used to build the model
####data.columns[numeric_vars].values


In [69]:
model_target = 'Run_Performance' ## Select target for classification

print("before splitting data")
#maintain class balance
X_train, X_test, y_train, y_test = train_test_split(numeric_x_data, y_data, test_size=0.25, stratify = y_data[model_target], random_state=42)

#split train set to create a pseudo test or validation dataset
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.33, stratify= y_train[model_target], random_state=42)


print('The training, validation and test data contain {}, {} and {} rows respectively'.format(len(X_train),len(X_validate),len(X_test)))

save_list_train = ['X_train','y_train','meta_data']
save_list_validate = ['X_validate','y_validate','meta_data']
save_list_test = ['X_test','y_test','meta_data']


before splitting data
The training, validation and test data contain 787, 389 and 392 rows respectively


In [75]:
import pandas as pd
# Create a Pandas Excel writer using XlsxWriter as the engine.

training_dataset_path=TRAINING_FILE_PATH 
validation_dataset_path=VALIDATION_FILE_PATH 
training_dataset=TRAINING_FILE_DIR 
validation_dataset=VALIDATION_FILE_DIR 
testing_dataset=TESTING_FILE_DIR 
input_file=INPUT_FILE 


model_target = 'Run_Performance' ## Select target for classification

print("before splitting data")
#maintain class balance
X_train, X_test, y_train, y_test = train_test_split(numeric_x_data, y_data, test_size=0.25, stratify = y_data[model_target], random_state=42)

#split train set to create a pseudo test or validation dataset
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.33, stratify= y_train[model_target], random_state=42)


print('The training, validation and test data contain {}, {} and {} rows respectively'.format(len(X_train),len(X_validate),len(X_test)))

save_list_train = ['X_train','y_train','meta_data']
save_list_validate = ['X_validate','y_validate','meta_data']
save_list_test = ['X_test','y_test','meta_data']



save_list_train = ['X_train','y_train','meta_data']
save_list_validate = ['X_validate','y_validate','meta_data']
save_list_test = ['X_test','y_test','meta_data']


for x in save_list_train:
    print("training_dataset", training_dataset)
    obj = pd.DataFrame(globals()[x])
    cmd = "obj.to_csv('{}/{}.csv')".format(training_dataset, x)
    print("cmd" , cmd)
    eval(cmd)

for y in save_list_validate:
    print("validation_dataset", validation_dataset)
    obj = pd.DataFrame(globals()[y])
    cmd = "obj.to_csv('{}/{}.csv')".format(validation_dataset, y)
    eval(cmd)  
    
for z in save_list_test:
    print("testing_dataset", testing_dataset)
    obj = pd.DataFrame(globals()[z])
    cmd = "obj.to_csv('{}/{}.csv')".format(testing_dataset, z)
    eval(cmd)

before splitting data
The training, validation and test data contain 787, 389 and 392 rows respectively
training_dataset gs://workshop_trial_artifact_store_pp/data/training
cmd obj.to_csv('gs://workshop_trial_artifact_store_pp/data/training/X_train.csv')
training_dataset gs://workshop_trial_artifact_store_pp/data/training
cmd obj.to_csv('gs://workshop_trial_artifact_store_pp/data/training/y_train.csv')
training_dataset gs://workshop_trial_artifact_store_pp/data/training
cmd obj.to_csv('gs://workshop_trial_artifact_store_pp/data/training/meta_data.csv')
validation_dataset gs://workshop_trial_artifact_store_pp/data/validation
validation_dataset gs://workshop_trial_artifact_store_pp/data/validation
validation_dataset gs://workshop_trial_artifact_store_pp/data/validation
testing_dataset gs://workshop_trial_artifact_store_pp/data/testing
testing_dataset gs://workshop_trial_artifact_store_pp/data/testing
testing_dataset gs://workshop_trial_artifact_store_pp/data/testing


In [60]:
import pandas as pd
# Create a Pandas Excel writer using XlsxWriter as the engine.


for x in save_list_train:
    print("training_dataset", training_dataset)
    obj = pd.DataFrame(globals()[x])
    #cmd = "obj.to_csv('../data/{}.csv')".format(x)
    #cmd = "obj.to_excel('/home/jupyter/aiplatform/data/{}.csv')".format(x)
    #writer_cmd = pd.ExcelWriter('{}.xlsx', engine='xlsxwriter')
    #cmd = "obj.to_excel(pd.ExcelWriter('{}/{}.xlsx', engine='xlsxwriter'), sheet_name='data')".format(training_dataset, x)
    eval(cmd)
    writer_cmd = "pd.ExcelWriter('{}/{}.xlsx', engine='xlsxwriter').save()".format(training_dataset, x)
    eval(writer_cmd)
    #eval(writer)
    #writer.save()
    #cmd = "obj.to_excel('/home/jupyter/aiplatform/data/{}.xlsx')".format(x)
    #cmd = "obj.to_csv('gs://workshop_trial_artifact_store_pp/data/training/{}.csv')".format(x)
    #eval(cmd)

    
for x in save_list_train:
        print("training_dataset", training_dataset)
        obj = pd.DataFrame(globals()[x])
        cmd = "obj.to_csv('{}/{}.xlsx')".format(training_dataset, x)
        eval(cmd)
    #    #cmd = "obj.to_csv('../data/{}.csv')".format(x)
    #    cmd = "obj.to_excel('{}/{}.xlsx')".format(training_dataset, x)
    #    eval(cmd)
    
    for x in save_list_validate:
        print("validation_dataset", validation_dataset)
        obj = pd.DataFrame(globals()[x])
        cmd = "obj.to_csv('{}/{}.csv')".format(validation_dataset, x)
        eval(cmd)    
    #    #cmd = "obj.to_csv('../data/{}.csv')".format(x)
    #    cmd = "obj.to_excel('{}/{}.xlsx')".format(validation_dataset, x)
    #    eval(cmd)    
    
    for x in save_list_test:
        print("testing_dataset", testing_dataset)
        obj = pd.DataFrame(globals()[x])
        cmd = "obj.to_csv('{}/{}.csv')".format(testing_dataset, x)
        eval(cmd)
    #    #cmd = "obj.to_csv('../data/{}.csv')".format(x)
    #    cmd = "obj.to_excel('{}/{}.xlsx')".format(testing_dataset, x)
    #    eval(cmd)    
        

    
    #df1.to_excel(writer, sheet_name='Sheet1')
#df2.to_excel(writer, sheet_name='Sheet2')
#df3.to_excel(writer, sheet_name='Sheet3')

# Close the Pandas Excel writer and output the Excel file.
#writer.save()

training_dataset gs://workshop_trial_artifact_store_pp/data/training


FileCreateError: [Errno 2] No such file or directory: 'gs://workshop_trial_artifact_store_pp/data/training/X_train.xlsx'

In [ ]:

    var_type_filter = [x in ['independent'] for x in meta_data['variable type']]
    var_dtype_filter = (data.dtypes == 'float64') | (data.dtypes == 'int64')

    numeric_vars = (var_type_filter & var_dtype_filter).values
    numeric_x_data = data[data.columns[numeric_vars]]

    #things to try to predict
    y_data = data[data.columns[(meta_data['target'] == 1).values]]

    #meta data about variables
    meta_data = meta_data.query('name in {}'.format(list(data.columns[numeric_vars].values))).set_index('name')
 
    #Variables which will be used to build the model
    ####data.columns[numeric_vars].values
        


    model_target = 'Run_Performance' ## Select target for classification
    
    print("before splitting data")
    #maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(numeric_x_data, y_data, test_size=0.25, stratify = y_data[model_target], random_state=42)

    #split train set to create a pseudo test or validation dataset
    X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.33, stratify= y_train[model_target], random_state=42)
    
    
    print('The training, validation and test data contain {}, {} and {} rows respectively'.format(len(X_train),len(X_validate),len(X_test)))

    save_list_train = ['X_train','y_train','meta_data']
    save_list_validate = ['X_validate','y_validate','meta_data']
    save_list_test = ['X_test','y_test','meta_data']

    for x in save_list_train:
        print("training_dataset", training_dataset)
        obj = pd.DataFrame(globals()[x])
        #cmd = "obj.to_csv('../data/{}.csv')".format(x)
        #cmd = "obj.to_csv('/home/jupyter/aiplatform/data/{}.csv')".format(x)
        cmd = "obj.to_excel('{}/{}.xlsx')".format(training_dataset, x)
        eval(cmd)
    
    for x in save_list_validate:
        print("validation_dataset", validation_dataset)
        obj = pd.DataFrame(globals()[x])
        #cmd = "obj.to_csv('../data/{}.csv')".format(x)
        #cmd = "obj.to_csv('/home/jupyter/aiplatform/data/{}.csv')".format(x)
        cmd = "obj.to_excel('{}/{}.xlsx')".format(validation_dataset, x)
        eval(cmd)    
    
    for x in save_list_test:
        print("testing_dataset", testing_dataset)
        obj = pd.DataFrame(globals()[x])
        #cmd = "obj.to_csv('../data/{}.csv')".format(x)
        #cmd = "obj.to_csv('/home/jupyter/aiplatform/data/{}.csv')".format(x)
        cmd = "obj.to_excel('{}/{}.xlsx')".format(testing_dataset, x)
        eval(cmd)  

In [17]:
import pandas as pd


# Create some Pandas dataframes from some data.
df1 = pd.DataFrame({'Data': [11, 12, 13, 14]})
df2 = pd.DataFrame({'Data': [21, 22, 23, 24]})
df3 = pd.DataFrame({'Data': [31, 32, 33, 34]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('my.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df1.to_excel(writer, sheet_name='Sheet1')
df2.to_excel(writer, sheet_name='Sheet2')
df3.to_excel(writer, sheet_name='Sheet3')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [49]:
from sklearn.ensemble import RandomForestClassifier
#simple sklearn impute and scale numeric pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

alpha = 'alpha'
max_iter = 2

classifier = RandomForestClassifier(
            n_estimators=3,
            max_depth=5,
            min_samples_leaf=2
 )

numeric_transformer = Pipeline([
  ('imputer', SimpleImputer(strategy='median')),
  ('scaler', StandardScaler()),
 ])


transform_list = []
  # If there exist numerical columns
transform_list.extend([
    ('numeric', numeric_transformer)
])

preprocessor = ColumnTransformer(transform_list)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#auto scale
scaler = StandardScaler()

estimator = Pipeline([
  ('imputer', imputer),
  ('scaler', scaler),
  ('classifier', classifier),
])

#prepare data for modeling
#use the pipeline created above
#_X_train = pipe.fit_transform(X_train)
#_y_train = y_train[model_target]    ## selected target label for prediction
#_X_test = pipe.fit_transform(X_validate)
#_y_test = y_validate[model_target]

#_X_train = pipe.fit_transform(X_train)
_X_train = X_train    
_y_train = y_train[model_target]    ## selected target label for prediction
_X_test = X_validate
_y_test = y_validate[model_target]

#print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))

estimator.set_params(classifier__n_estimators=5, classifier__max_depth=10, classifier__min_samples_leaf=2 ) 
#pipeline.fit(X_train, y_train)
estimator.fit(_X_train, _y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(max_depth=10, min_samples_leaf=2,
                                        n_estimators=5))])